# Classifier

1. Logistic regression (https://www.datacamp.com/community/tutorials/understanding-logistic-regression-python)


functions:
https://scikit-learn.org/stable/modules/feature_selection.html

example:
https://scikit-learn.org/stable/auto_examples/feature_selection/plot_feature_selection.html#sphx-glr-auto-examples-feature-selection-plot-feature-selection-py

tut:
https://medium.com/@GouthamPeri/pipeline-with-tuning-scikit-learn-b2789dca9dc2

https://stackoverflow.com/questions/33376078/python-feature-selection-in-pipeline-how-determine-feature-names


https://scikit-learn.org/stable/auto_examples/model_selection/grid_search_text_feature_extraction.html#sphx-glr-auto-examples-model-selection-grid-search-text-feature-extraction-py

https://scikit-learn.org/stable/auto_examples/svm/plot_svm_anova.html#sphx-glr-auto-examples-svm-plot-svm-anova-py

## Import packages

In [1]:
import pandas as pd
from tqdm.notebook import tqdm
from time import time
import random
random.seed(32)
import sklearn
# from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.metrics import f1_score, classification_report, precision_recall_curve, roc_auc_score, plot_roc_curve, confusion_matrix, plot_confusion_matrix
from sklearn.dummy import DummyClassifier
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Import data

In [2]:
df = pd.read_pickle("allfeat_df_allcomments.pkl")
df.info()


# allfeat_df_allcomments
# wordlists_lin_feat_df_withoutuserfeat_allcomments

<class 'pandas.core.frame.DataFrame'>
Index: 1606 entries, -Areopagan- to zyzee
Columns: 18201 entries, ('text', 'body') to ('lda', 'ldahundred')
dtypes: float64(2091), int64(16103), object(7)
memory usage: 223.0+ MB


In [3]:
df.head()

text  \
                                                           body   
author                                                            
-Areopagan-   Your first and second question is the same que...   
-BigSexy-     I've been asked to cum everywhere with my ex j...   
-BlitzN9ne    I'm currently in the middle of making a Payday...   
-CrestiaBell  First and foremost I extend my condolences to ...   
-dyad-        I failed both...I'm great at reading people ir...   

                                                                 \
                                                       doc_body   
author                                                            
-Areopagan-   [Your first and second question is the same qu...   
-BigSexy-     [I've been asked to cum everywhere with my ex ...   
-BlitzN9ne    [I'm currently in the middle of making a Payda...   
-CrestiaBell  [First and foremost I extend my condolences to...   
-dyad-        [I failed both...I'm great at reading people i...   

                                                           data       post  \
                                                            utc      score   
author                                                                       
-Areopagan-   [1513882848, 1513744846, 1522253427, 151370438...   2.000000   
-BigSexy-     [1507650565, 1516397088, 1502590403, 151682490...   4.266714   
-BlitzN9ne    [1422166355, 1423504286, 1449881503, 145521567...   9.644956   
-CrestiaBell  [1462304635, 1528773104, 1513663029, 148131600...  24.890662   
-dyad-        [1475875524, 1473096864, 1505168466, 150318014...   7.234043   

                                            subreddit          time  \
             controversiality    gilded num_subreddit     mean_time   
author                                                                
-Areopagan-          0.000000  0.000000             1  2.137261e+06   
-BigSexy-            0.020737  0.000000           147  1.003843e+04   
-BlitzN9ne           0.014159  0.000000           116  4.830648e+04   
-CrestiaBell         0.017687  0.000866           149  1.220542e+04   
-dyad-               0.000000  0.000000             5  3.799737e+05   

                                   ...      psych                        \
             median_time max_time  ...    valence    arousal  dominance   
author                             ...                                    
-Areopagan-     893447.0  6721687  ...   0.067701   0.047306   0.066281   
-BigSexy-          760.0  1292061  ...  11.312668   7.874620  11.121217   
-BlitzN9ne         793.5  3039780  ...   5.300178   3.868446   5.229659   
-CrestiaBell      1365.0   594290  ...  29.105901  20.767584  28.330990   
-dyad-           57538.0  6062289  ...   1.376648   0.948765   1.345483   

              lin_feat                         psych                lda  \
             negations  articles    future mrc_cmean mrc_pmean ldafifty   
author                                                                    
-Areopagan-   3.200000  3.600000  0.400000       0.0       0.0       45   
-BigSexy-     0.716124  1.496246  0.133000       0.0       0.0        9   
-BlitzN9ne    0.245310  0.523186  0.060885       0.0       0.0       26   
-CrestiaBell  0.479159  1.213482  0.076933       0.0       0.0        8   
-dyad-        1.051064  1.765957  0.076596       0.0       0.0       12   

                         
             ldahundred  
author                   
-Areopagan-           4  
-BigSexy-            42  
-BlitzN9ne           99  
-CrestiaBell         60  
-dyad-               85  

[5 rows x 18201 columns]

In [4]:
# predictors = df.columns
# i = 0
# predictorsfile=open('featurelist.txt','w')
# # predictorsfile.writelines(predictors)
# for index in range(len(predictors)):
#     predictorsfile.write(str(i))
#     predictorsfile.write(predictors[index][0])
#     predictorsfile.write(" ")
#     predictorsfile.write(predictors[index][1])
#     predictorsfile.write('\n')
#     i+=1
# predictorsfile.close()

In [5]:
# # multilevel columns

# lst1 = (5)*["data"]
# lst9 = (15-5)*["traits"]
# lst10 = ["data"]
# lst2 = (21-17)*["global"]
# lst3 = (45-21)*["time"]
# lst4 = (16103-45)*["subreddits"]
# lst5 = (16116-16103)*["extra_features"]
# lst6 = (96308-16116)*["word_ngrams"]
# lst7 = (103829-96308)*["char_ngrams"]
# lst8 = (103889-103829)*["wordlists"]
# lst11 = 2 * ["lda"]
# headers = lst1 + lst9  + lst10 + lst2 + lst3 + lst4 +lst5 + lst6 + lst7 + lst8 + lst11
# columns = df.columns.values
# print(len(headers))
# print(len(columns))
# arrays = [headers] + [columns]
# df.columns=pd.MultiIndex.from_arrays(arrays)
# df.head()

In [6]:
# test = df['subreddits']
# test
# i=0
# for i in range(len(test)):
#     print(i)
#     print(test.iloc[:, i].to_numpy().nonzero())

In [7]:
# df['traits', 'agree5'] = df['traits', 'agreeableness'].apply(lambda x: 0 if x<20 else(1 if x>19 and x<40 else(2 if x>39 and x<60 else(3 if x>59 and x<80 else 4))))
# df['traits', 'openn5'] = df['traits', 'openness'].apply(lambda x: 0 if x<20 else(1 if x>19 and x<40 else(2 if x>39 and x<60 else(3 if x>59 and x<80 else 4))))
# df['traits', 'consc5'] = df['traits', 'conscientiousness'].apply(lambda x: 0 if x<20 else(1 if x>19 and x<40 else(2 if x>39 and x<60 else(3 if x>59 and x<80 else 4))))
# df['traits', 'extra5'] = df['traits', 'extraversion'].apply(lambda x: 0 if x<20 else(1 if x>19 and x<40 else(2 if x>39 and x<60 else(3 if x>59 and x<80 else 4))))
# df['traits', 'neuro5'] = df['traits', 'neuroticism'].apply(lambda x: 0 if x<20 else(1 if x>19 and x<40 else(2 if x>39 and x<60 else(3 if x>59 and x<80 else 4))))

In [8]:
# df = df.dropna(axis=0, how='all')
# df = df.dropna(axis=1, how='all')

In [9]:
featurelst = []
for i in range(len(predictors)):
    featurelst.append(predictors[i][0])
featureset = set(featurelst)
print(featureset)

{'text', 'lda', 'post', 'trait', 'data', 'time', 'x_feat', 'lin_feat', 'empath', 'psych', 'subreddit', 'ngram'}


In [10]:
log_wordngrams_df = df[['trait', 'ngram']]
log_charngrams_df = df[['trait', 'ngram']]
log_wordlists_df =  df[['trait', 'x_feat', 'lin_feat', 'psych', 'empath']]
log_posts_df = df[['trait', 'post', 'time', 'subreddit', 'lda']]
log_postswithoutsubreddits_df = df[['trait', 'post', 'time', 'lda']]

In [11]:
droplst = []
index = 17124
for i in range(1000):
    index = index+1
    droplst.append(predictors[index][1])

log_wordngrams_df = log_wordngrams_df.drop(droplst, axis=1, level=1)

In [12]:
worddroplst = []
wordindex = 16125
for i in range(1000):
    wordindex = wordindex+1
    worddroplst.append(predictors[wordindex][1])

log_charngrams_df = log_charngrams_df.drop(worddroplst, axis=1, level=1)
log_charngrams_df

trait                                          \
             agreeableness openness conscientiousness extraversion   
author                                                               
-Areopagan-            0.0     99.0              96.0         60.0   
-BigSexy-             39.0     92.0               1.0         18.0   
-BlitzN9ne            50.0     85.0              15.0         50.0   
-CrestiaBell          50.0     85.0              50.0         85.0   
-dyad-                60.0     67.0              45.0         10.0   
...                    ...      ...               ...          ...   
zugzwang_03           10.0     41.0              86.0         83.0   
zuluthrone            17.0     96.0              28.0         95.0   
zwelg                 39.0     89.0              91.0         80.0   
zymmaster             28.0     47.0              62.0         21.0   
zyzee                 88.0     78.0              31.0         75.0   

                                                             ...     ngram  \
             neuroticism big5_a big5_o big5_c big5_e big5_n  ...        y    
author                                                       ...             
-Areopagan-          1.0      0      1      1      1      0  ...  0.027200   
-BigSexy-            4.0      0      1      0      0      0  ...  0.043505   
-BlitzN9ne          30.0      1      1      0      1      0  ...  0.055295   
-CrestiaBell        50.0      1      1      1      1      1  ...  0.045581   
-dyad-              47.0      1      1      0      0      0  ...  0.049842   
...                  ...    ...    ...    ...    ...    ...  ...       ...   
zugzwang_03         18.0      0      0      1      1      0  ...  0.054066   
zuluthrone          34.0      0      1      0      1      0  ...  0.040346   
zwelg                3.0      0      1      1      1      0  ...  0.000000   
zymmaster           49.0      0      0      1      0      0  ...  0.049808   
zyzee               10.0      1      1      0      1      0  ...  0.060042   

                                                                          \
                   y s   ye_char       yea        yo       yon        yp   
author                                                                     
-Areopagan-   0.000000  0.020915  0.021700  0.007215  0.007525  0.000000   
-BigSexy-     0.004567  0.012574  0.004844  0.007757  0.002873  0.003249   
-BlitzN9ne    0.004506  0.019322  0.009267  0.009527  0.006000  0.010355   
-CrestiaBell  0.005556  0.012763  0.005753  0.012696  0.007740  0.003121   
-dyad-        0.006174  0.013465  0.005015  0.006789  0.002236  0.016283   
...                ...       ...       ...       ...       ...       ...   
zugzwang_03   0.006181  0.014229  0.004873  0.005862  0.004105  0.003795   
zuluthrone    0.002921  0.009208  0.004398  0.011042  0.005837  0.007990   
zwelg         0.000000  0.012877  0.013360  0.000000  0.000000  0.027605   
zymmaster     0.006375  0.019120  0.011743  0.006654  0.003903  0.007465   
zyzee         0.003184  0.025228  0.006282  0.015666  0.013070  0.002163   

                                            
                   ype        yt       yth  
author                                      
-Areopagan-   0.000000  0.000000  0.000000  
-BigSexy-     0.001611  0.004111  0.003562  
-BlitzN9ne    0.007065  0.006066  0.005993  
-CrestiaBell  0.001219  0.006251  0.006028  
-dyad-        0.015979  0.008390  0.008055  
...                ...       ...       ...  
zugzwang_03   0.002287  0.004201  0.003870  
zuluthrone    0.003213  0.005250  0.004340  
zwelg         0.000000  0.000000  0.000000  
zymmaster     0.006780  0.006262  0.005966  
zyzee         0.002335  0.012794  0.009632  

[1606 rows x 1014 columns]

In [13]:
def all_hist_true(df):
    plt.figure(figsize = (16, 16))
#     plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
    plt.subplot(3, 2, 1)
    plt.hist(df['trait', 'openness'], bins = 20)
    plt.title('Agreeableness')
    
    plt.subplot(3, 2, 2)
    plt.hist(df['trait', 'conscientiousness'], bins = 20)
    plt.title('Openness')
    
    plt.subplot(3, 2, 3)
    plt.hist(df['trait', 'extraversion'], bins = 20)
    plt.title('Conscientiousness')
    
    plt.subplot(3, 2, 4)
    plt.hist(df['trait', 'agreeableness'], bins = 20)
    plt.title('Extraversion')
    
    plt.subplot(3, 2, 5)
    plt.hist(df['trait', 'neuroticism'], bins = 20)
    plt.title('Neuroticism')
    
    plt.suptitle("Histograms of the true trait values")
    plt.subplots_adjust(left=0.1, 
                    bottom=0.1,  
                    right=0.9,  
                    top=0.9,  
                    wspace=0.4,  
                    hspace=0.4) 
    plt.show()

## Trait

In [14]:
#split dataset in features and target variable depending on which trait to focus on
def trait(df, classes, trait_name):
    featuredf = df.drop(['data', 'trait', 'text'], axis=1, level=0)
    feature_cols = featuredf.columns.tolist()
    
    x = df[feature_cols] 
    
    if classes=='binary':
    
        if trait_name == 'agreeableness':
            y = df['trait', 'big5_o']
        elif trait_name == 'openness':
            y = df['trait', 'big5_c']
        elif trait_name == 'conscientiousness':
            y = df['trait', 'big5_e']
        elif trait_name == 'extraversion':
            y = df['trait', 'big5_a']
        elif trait_name == 'neuroticism':
            y = df['trait', 'big5_n']   
    elif classes=='multi':
        if trait_name == 'agreeableness':
            y = df['trait', 'big5_o_multi']
        elif trait_name == 'openness':
            y = df['trait', 'big5_c_multi']
        elif trait_name == 'conscientiousness':
            y = df['trait', 'big5_e_multi']
        elif trait_name == 'extraversion':
            y = df['trait', 'big5_a_multi']
        elif trait_name == 'neuroticism':
            y = df['trait', 'big5_n_multi'] 
    elif classes=='linear':
        if trait_name == 'agreeableness':
            y = df['trait', 'agreeableness']
        elif trait_name == 'openness':
            y = df['trait', 'openness']
        elif trait_name == 'conscientiousness':
            y = df['trait', 'conscientiousness']
        elif trait_name == 'extraversion':
            y = df['trait', 'extraversion']
        elif trait_name == 'neuroticism':
            y = df['trait', 'neuroticism']  
    return x,y 

## Classifier

In [15]:
# ANOVA feature selection for numeric input and categorical output

def create_pipeline(x_train, y_train, classifier, num_feat, weighted):
    if weighted==True: 
        if classifier == "log":
            pipeline = Pipeline([
              ('variance_threshold', VarianceThreshold()),
              ('scaler', StandardScaler()),
              ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
              ('classification', LogisticRegression(class_weight='balanced', max_iter = 200, n_jobs=-1))
            ])
        elif classifier == "multilog":
            pipeline = Pipeline([
              ('variance_threshold', VarianceThreshold()),
              ('scaler', StandardScaler()),
              ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
              
              ('classification', LogisticRegression(class_weight='balanced', multi_class='multinomial', 
                                                    max_iter = 200, solver='lbfgs', n_jobs=-1))
            ])
        elif classifier == "svm":
            pipeline = Pipeline([
              ('variance_threshold', VarianceThreshold()),
              ('scaler', StandardScaler()),
              ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
              ('classification', svm.SVC(class_weight='balanced', max_iter = 1000))
            ])

    else:
        if classifier == "log":
            pipeline = Pipeline([
              ('variance_threshold', VarianceThreshold()),
              ('scaler', StandardScaler()),
              ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
              ('classification', LogisticRegression(max_iter = 200, n_jobs=-1))
            ])
        elif classifier == "multilog":
            pipeline = Pipeline([
              ('variance_threshold', VarianceThreshold()),
              ('scaler', StandardScaler()),
              ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
              ('classification', LogisticRegression(multi_class='multinomial', max_iter = 200, solver='lbfgs', 
                                                    n_jobs=-1))
            ])
        elif classifier == "mlp":
            pipeline = Pipeline([
              ('variance_threshold', VarianceThreshold()),
              ('scaler', StandardScaler()),
              ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
              ('classification', MLPClassifier(hidden_layer_sizes=(3,)))
            ])
        elif classifier == "svm":
            pipeline = Pipeline([
              ('variance_threshold', VarianceThreshold()),
              ('scaler', StandardScaler()),
              ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
              ('classification', svm.SVC(max_iter = 1000))
            ])
        elif classifier == "svmlinear":
            pipeline = Pipeline([
              ('variance_threshold', VarianceThreshold()),
              ('scaler', StandardScaler()),
              ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
              ('classification', svm.LinearSVC(max_iter = 1000))
            ])
        elif classifier == "knn":
            pipeline = Pipeline([
              ('variance_threshold', VarianceThreshold()),
              ('scaler', StandardScaler()),
              ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
              ('classification', KNeighborsClassifier(n_neighbors=1, n_jobs=-1))
            ])
        elif classifier == "linear":
            pipeline = Pipeline([
              ('variance_threshold', VarianceThreshold()),
              ('scaler', StandardScaler()),
              ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
              ('classification', LinearRegression(n_jobs=-1))
            ])
    return pipeline

## Get names of 30 most important features

In [16]:
# get names of the features
def get_names(x, pipeline):
    features = pipeline.named_steps['feature_selection']
    names = x.columns[features.get_support(indices=True)]
    return names
# names = get_names(logpipe)
# print(names)

### Get p-values

In [17]:
def get_pvalues(pipeline, x):
#     x_indices = np.arange(x.shape[-1])
#     selector = SelectKBest(f_classif, k=30)
#     selector.fit(x_train, y_train)
#     scores = -np.log10(selector.pvalues_)
    features = pipeline.named_steps['feature_selection']
    pvalues = features.pvalues_
#     pvalues /= pvalues.max()
    dfpvalues = pd.DataFrame(features.pvalues_)
    dfscores = pd.DataFrame(features.scores_)
    dfcolumns = pd.DataFrame(x.columns)
    #concat two dataframes for better visualization 
    featureScores = pd.concat([dfcolumns,dfscores, dfpvalues],axis=1)
    featureScores.columns = ['specs','score', 'pvalue']
    featureScores.sort_values(by='pvalue')

    plt.figure(figsize = (12, 4))
    plt.subplot(1, 2, 1)
    plt.hist(pvalues, bins=20)
    plt.title('All p-values')
    plt.subplot(1, 2, 2)
    smallpvalues = pvalues[pvalues<0.1]
    plt.hist(smallpvalues, bins=10)
    plt.title('Small p-values')
    
    plt.suptitle("Histograms of the p-values")
    plt.subplots_adjust(left=0.1, 
                    bottom=0.1,  
                    right=0.9,  
                    top=0.9,  
                    wspace=0.4,  
                    hspace=0.4) 
    plt.show()
    
    return featureScores

## Scores

In [18]:
def scores(y_test, y_pred, presentationtype):
    
    if presentationtype == "scores":
        accuracy=metrics.accuracy_score(y_test, y_pred)
        precision=metrics.precision_score(y_test, y_pred)
        recall=metrics.recall_score(y_test, y_pred)
        f_one=metrics.f1_score(y_test, y_pred)
        return accuracy, precision, recall, f_one
    if presentationtype == "report":
        report = classification_report(y_test, y_pred)
        return report


def score_plot(logreg, y_test, x_test):
    lr_probs = logreg.predict_proba(x_test)
    # keep probabilities for the positive outcome only
    lr_probs = lr_probs[:, 1]
    # predict class values
#     yhat = logreg.predict(x_test)
    lr_precision, lr_recall, _ = precision_recall_curve(y_test, lr_probs)
#     lr_f1, lr_auc = f1_score(y_test, yhat), auc(lr_recall, lr_precision)

    # plot the precision-recall curves
    no_skill = len(y_test[y_test==1]) / len(y_test)
    fig, ax = plt.subplots()
    plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
    plt.plot(lr_recall, lr_precision, marker='.', label='Classifier')
    # axis labels
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    # show the legend
    plt.legend()
    plt.show()
    
    return lr_precision, lr_recall


### Score plot
Smaller values on the x-axis of the plot indicate lower false positives and higher true negatives.


Larger values on the y-axis of the plot indicate higher true positives and lower false negatives.

## Create and print confusion matrix

In [19]:
def create_cnfmatrix(clf, x_test, y_test, y_pred, plotting, detailed):
    cnfpipe_matrix = confusion_matrix(y_test, y_pred)
#     print(cnfpipe_matrix)
#     disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cnfpipe_matrixcmap=plt.cm.Blues, normalize=normalize)
#     disp.plot() 
    if detailed:
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        sumpositive = tp + fn
        sumnegative = fp + tn
        sumcorrect = tp + tn
        sumwrong = fp + fn
        sumall = tn+fp+fn+tp
        print("TN, FP, FN, TP: ", tn, fp, fn, tp, "\nSum: ", sumall, "\nSum correct predictions: ", 
              sumcorrect, "Percent: ", sumcorrect/sumall, "\nSum wrong predictions: ", sumwrong, "\tPercent: ",
              sumwrong/sumall, "\nSum actual positives: ", sumpositive, "\tPercent: ", sumpositive/sumall,
              "\nSum actual negatives: ", sumnegative, "\tPercent: ", sumnegative/sumall)

    if plotting:
#         %matplotlib inline
#         class_names=[0,1] # name  of classes
#         fig, ax = plt.subplots()
#         tick_marks = np.arange(len(class_names))
#         plt.xticks(tick_marks, class_names)
#         plt.yticks(tick_marks, class_names)
#         # create heatmap
#         sns.heatmap(pd.DataFrame(cnfpipe_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
#         ax.xaxis.set_label_position("bottom")
#         disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cnfpipe_matrix, cmap=plt.cm.Blues, normalize=normalize)
#         disp.plot() 
        plot_confusion_matrix(clf, x_test, y_test,
                                 cmap=plt.cm.Blues,
                                 normalize='true')
#         disp.ax_.set_title('Confusion matrix')
        plt.tight_layout()
        plt.title('Confusion matrix', y=1.1)
        plt.ylabel('Actual label')
        plt.xlabel('Predicted label')
#         disp.plot() 
        plt.show()
        
# cnfmatrix = create_cnfmatrix(y_test, y_pred, True)

## ROC Curve

In [20]:
def roc_aucscore(clf, x_test, y_test, classes, plotting, detailed):
    if detailed:
        print(roc_auc_score(y, clf.predict_proba(x), multi_class='ovo'))
        return score
    
    if plotting and classes == 'binary':
        plot_roc_curve(clf, x_test, y_test)
        plt.title('ROC Curve', y=1.1)
        plt.show()

# Wrapper


nested stratified cv:
https://scikit-learn.org/stable/auto_examples/model_selection/plot_nested_cross_validation_iris.html#sphx-glr-auto-examples-model-selection-plot-nested-cross-validation-iris-py

https://weina.me/nested-cross-validation/


https://towardsdatascience.com/nested-cross-validation-hyperparameter-optimization-and-model-selection-5885d84acda
https://gist.github.com/krsatyam1996/9640ed8baa20d3dc11822564710a8d71#file-nested_cv-py

In [21]:
traits = ['openness', 'conscientiousness', 'extraversion', 'agreeableness', 'neuroticism']

def switching(trait):
    switcher={
            'openness':30,
            'conscientiousness': 30,
            'agreeableness': 30,
            'extraversion': 30,
            'neuroticism':30
         }
    return switcher.get(trait,"Invalid")

In [22]:
def classify(df, classifier, classes, trainscores=False, plotting = False, weighted = False, detailed=False):
    for trait_name in traits:
        num_feat = switching(trait_name)
        print("Trait to predict: ", trait_name)
        x,y = trait(df, classes,trait_name)
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)
#         scores = cross_val_scores(pipeline,X_train,y_train,cv=5,scoring='f1_macro')
        if trainscores:
            x_test = x_train
            y_test = y_train
        if detailed: 
            print("Number of authors in y_train: ", len(y_train))
            print("Number of authors in y_test: ", len(y_test))
        clf = create_pipeline(x_train, y_train, classifier, num_feat, weighted)
        clf.fit(x_train, y_train)
        y_pred=clf.predict(x_test)
        if classes=='linear':
            print("Score (Reg: r sqared, SVM: accuracy): ", clf.score(x_test, y_test))
#             print(y_pred)    
        else:
            if detailed:
                print("Number of authors in y_pred: ", len(y_pred))
                names = get_names(x, clf)
                print("Names of the top", len(names), "features: \n", names, "\n")
                pvalues = get_pvalues(clf, x)
            #     print("p-values of", len(pvalues), "features: \n", pvalues, "\n")
                if trait_name=="openness":
                    count = pvalues['pvalue'].le(0.02).sum()
                if trait_name=="conscientiousness":
                    count = pvalues['pvalue'].le(0.07).sum()
                if trait_name=="extraversion":
                    count = pvalues['pvalue'].le(0.05).sum()
                if trait_name=="agreeableness":
                    count = pvalues['pvalue'].le(0.04).sum()
                if trait_name=="neuroticism":
                    count = pvalues['pvalue'].le(0.04).sum()
                print("Number of features with this threshold: ", count)
        #         print("\nP-Values: \nNumber of features: ", 30)
        #         print(pvalues.nsmallest(count,'pvalue'))
                print("\n")
            report = scores(y_test, y_pred, "report")
            print("Classification report: \n", report)
            if plotting: 
                cnfmatrix = create_cnfmatrix(clf, x_test, y_test, y_pred, plotting, detailed) 
                rocplot = roc_aucscore(clf, x_test, y_test, classes, plotting, detailed)
        #     accuracy, precision, recall, f_one = scores(y_test, y_pred, "scores")
        #     print("Scores:\nAccuracy:",accuracy, "\nPrecision:",precision, "\nRecall:",recall, "\nF1 score:",f_one)
                if (classifier == 'log' and classes == 'binary'):
                    lr_precision, lr_recall = score_plot(clf, y_test, x_test)
            plt.show()
    #     print("Scores:\nLR_Precision:",lr_precision, "\nLR_Recall:",lr_recall)

## Wrapper with nested stratified cross validation

In [30]:
def create_pipeline_cv(classifier, num_feat):
    if classifier == "mcc":
        pipeline = Pipeline([
          ('variance_threshold', VarianceThreshold()),
          ('scaler', StandardScaler()),
          ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
          ('classification', DummyClassifier(strategy="most_frequent"))
        ])
    if classifier == "log":
        pipeline = Pipeline([
          ('variance_threshold', VarianceThreshold()),
          ('scaler', StandardScaler()),
          ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
          ('classification', LogisticRegression(class_weight='balanced'))
        ])
    elif classifier == "multilog":
        pipeline = Pipeline([
          ('variance_threshold', VarianceThreshold()),
          ('scaler', StandardScaler()),
          ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
          ('classification', LogisticRegression(multi_class='multinomial', n_jobs=-1))
        ])
    elif classifier == "mlp":
        pipeline = Pipeline([
          ('variance_threshold', VarianceThreshold()),
          ('scaler', StandardScaler()),
          ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
          ('classification', MLPClassifier())
        ])
#         n_layers_=3
    elif classifier == "svm":
        pipeline = Pipeline([
          ('variance_threshold', VarianceThreshold()),
          ('scaler', StandardScaler()),
          ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
          ('classification', svm.SVC(class_weight='balanced', probability=True))
        ])
    elif classifier == "svmlinear":
        pipeline = Pipeline([
          ('variance_threshold', VarianceThreshold()),
          ('scaler', StandardScaler()),
          ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
          ('classification', svm.LinearSVC())
        ])
    elif classifier == "knn":
        pipeline = Pipeline([
          ('variance_threshold', VarianceThreshold()),
          ('scaler', StandardScaler()),
          ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
          ('classification', KNeighborsClassifier(n_neighbors=1, n_jobs=-1))
        ])
    elif classifier == "linear":
        pipeline = Pipeline([
          ('variance_threshold', VarianceThreshold()),
          ('scaler', StandardScaler()),
          ('feature_selection',  SelectKBest(f_classif, k=num_feat)),
          ('classification', LinearRegression(n_jobs=-1))
        ])
    return pipeline

In [31]:
def get_params(classifier):
    if classifier == 'log':
        params = {'classification__solver': ['lbfgs', 'liblinear', 'saga'], 
                  'classification__max_iter': [100, 200, 500, 1000],
                  'classification__C': [10**x for x in range(-3,5)]}
    if classifier == 'multilog':
        params = {'classification__class_weight': [None, 'balanced'], 
                  'classification__solver': ['lbfgs', 'saga'], 
                  'classification__max_iter': [100, 200, 500, 1000]}
    elif classifier == 'mlp':
        params = {'classification__hidden_layer_sizes': [(50,), (100,), (200,), (500,)]}
    elif classifier == 'svm':
        params = {'classification__kernel': ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],
                  'classification__gamma': ['scale', 'auto'], 
                  'classification__max_iter': [100, 200, 500, 1000],
                  'classification__C': [10**x for x in range(-3,5)]}
    elif classifier == 'mcc':
        params = {}
    return params
# 'classification__class_weight': ['balanced'], 

def classify_cv(df, classes, clf_lst):
    for option in tqdm(clf_lst):
        print("Classifier: ", option, "\n")
        for trait_name in tqdm(traits):
            num_feat = switching(trait_name)
            print("\nTrait to predict: ", trait_name, "\n")
            x,y = trait(df, classes, trait_name)          
            
            cv_outer = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
            cv_outer_lst = cv_outer.split(x, y)

            f1macro_lst = []
            for train_idx, val_idx in tqdm(cv_outer_lst):
                train_data, val_data = x.iloc[train_idx], x.iloc[val_idx]
                train_target, val_target = y.iloc[train_idx], y.iloc[val_idx]

                print("\n\tCreate pipeline with", option, "...")
                clf = create_pipeline_cv(option, num_feat)
                if option == 'log':
                    cv_inner = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
                if option == 'svm' or option == 'mlp' or option == 'mcc':
                    cv_inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
                params = get_params(option)
                print("\tStart grid search...")
                t0 = time()
                gd_search = GridSearchCV(clf, params, scoring='f1_macro', n_jobs=-1, cv=cv_inner).fit(train_data, train_target)
                print("\tGrid search done in %0.3fs" % (time() - t0))
                print("\tGet best model...")
                best_model = gd_search.best_estimator_
                print(best_model)

                print("\tFit best model...")
                clfnew = best_model.fit(train_data, train_target)
                y_pred = clfnew.predict(val_data)
                f1_macro = f1_score(val_target, y_pred, average='macro')
                f1macro_lst.append(f1_macro)
                print("Val Acc:",f1_macro , "Best GS Acc:",gd_search.best_score_, "Best Params:",gd_search.best_params_)

              # Training final model
            f1macro_avg = np.mean(f1macro_lst)
            print("Average f1 macro score: ", f1macro_avg)


In [28]:
# if option == 'log':
#                 inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
#             if option == 'svm' or option == 'mlp':
#                 inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
#             outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

#             pipeline = create_pipeline_cv(X_train, y_train, option, num_feat)
#             if option == 'log' or option == 'svm' or option=='mlp':
#                 t0 = time()
#                 print("\tStart grid search (inner cv)...")
#                 params = get_params(option)
#                 clf = GridSearchCV(estimator=pipeline, param_grid=params, scoring='f1_macro', n_jobs=-1, cv=inner_cv)
#                 clf.fit(X_train, y_train)
#                 print("\tGrid search done in %0.3fs" % (time() - t0))

#                 best_model = clf.best_estimator_
#                 print("\tBest model: ", best_model)
#                 t1 = time()
#                 print("\tStart cross validation (outer)...")
#                 nested_score = cross_val_score(clf, X=X_train, y=y_train, scoring='f1_macro', n_jobs=-1, cv=outer_cv)
#                 print("\tCross_val_score calculated in %0.3fs" % (time() - t1))
#                 score = nested_score.mean()
#                 print("Score: ", score)
#             else:
#                 best_model = pipeline
#                 t0 = time()
#                 print("\tStart cross validation...")
#                 nested_score = cross_val_score(pipeline, X=X_train, y=y_train, scoring='f1_macro', n_jobs=-1, cv=outer_cv)
#                 print("\tCross_val_score calculated in %0.3fs" % (time() - t0))
#                 score = nested_score.mean()
#                 print("Score: ", score)
            
#             print("Computing final model...")
#             model = best_model.fit(X_train, y_train)
#             names = get_names(x, model)
#             print("Names of the top", len(names), "features: \n", names, "\n")
#             y_pred_prob = model.predict_proba(X_test)[:,1]
#             y_pred = model.predict(X_test)
#             print("AUC", roc_auc_score(y_test, y_pred_prob))
#             report = scores(y_test, y_pred, "report")
#             print("Classification report: \n", report) 
#             cnfmatrix = create_cnfmatrix(model, X_test, y_test, y_pred, plotting=True, detailed=False) 
#             rocplot = roc_aucscore(model, X_test, y_test, classes, plotting=True, detailed=False)
#             plt.show()

## Methodological replication

In [32]:
clf_lst = ['mcc', 'log', 'mlp', 'svm']
classify_cv(df, 'binary', clf_lst)

  0%|          | 0/4 [00:00<?, ?it/s]

Classifier:  mcc 



  0%|          | 0/5 [00:00<?, ?it/s]


Trait to predict:  openness 



0it [00:00, ?it/s]

...

/home/sophia/ma_py/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


Val Acc: 0.5856536972155634 Best GS Acc: 0.5684823961551178 Best Params: {'classification__C': 1, 'classification__gamma': 'scale', 'classification__kernel': 'sigmoid', 'classification__max_iter': 1000}

	Create pipeline with svm ...
	Start grid search...


/home/sophia/ma_py/.venv/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.3320791  0.37931844 0.35718906 0.58887762 0.33072917 0.33072917
 0.33072917 0.333417   0.33072917 0.33072917 0.37456486 0.34443483
 0.33591731 0.33591731 0.33591731 0.33591731        nan        nan
        nan        nan 0.3320791  0.37931844 0.35718906 0.58887762
 0.33072917 0.33072917 0.33072917 0.333417   0.33072917 0.33072917
 0.37456486 0.34443483 0.33591731 0.33591731 0.33591731 0.33591731
        nan        nan        nan        nan 0.39613466 0.50259057
 0.54297341 0.58250613 0.33072917 0.33072917 0.33072917 0.39691089
 0.33072917 0.33072917 0.33072917 0.33557323 0.33591731 0.33591731
 0.33591731 0.36492807        nan        nan        nan        nan
 0.39613466 0.50259057 0.54297341 0.58250613 0.33072917 0.33072917
 0.33072917 0.39691089 0.33072917 0.33072917 0.33072917 0.33557323
 0.33591731 0.33591731 0.33591731 0.3649280

	Grid search done in 347.918s
	Get best model...
Pipeline(steps=[('variance_threshold', VarianceThreshold()),
                ('scaler', StandardScaler()),
                ('feature_selection', SelectKBest(k=30)),
                ('classification',
                 SVC(C=0.001, class_weight='balanced', kernel='linear',
                     max_iter=1000, probability=True))])
	Fit best model...
Val Acc: 0.45675603455954394 Best GS Acc: 0.5888776195024077 Best Params: {'classification__C': 0.001, 'classification__gamma': 'scale', 'classification__kernel': 'linear', 'classification__max_iter': 1000}
Average f1 macro score:  0.5265893412241617


/home/sophia/ma_py/.venv/lib/python3.8/site-packages/sklearn/svm/_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


In [33]:
small_clf_lst = ['log']
print("Logistic regression with subsets of the features")
print("LR with word ngrams")
classify_cv(log_wordngrams_df, 'binary', small_clf_lst)

Logistic regression with subsets of the features
LR with word ngrams


  0%|          | 0/1 [00:00<?, ?it/s]

Classifier:  log 



  0%|          | 0/5 [00:00<?, ?it/s]


Trait to predict:  openness 



...


	Create pipeline with log ...
	Start grid search...
	Grid search done in 13.146s
	Get best model...
Pipeline(steps=[('variance_threshold', VarianceThreshold()),
                ('scaler', StandardScaler()),
                ('feature_selection', SelectKBest(k=30)),
                ('classification',
                 LogisticRegression(C=1, class_weight='balanced'))])
	Fit best model...
Val Acc: 0.5357880381842977 Best GS Acc: 0.5797161814074788 Best Params: {'classification__C': 1, 'classification__max_iter': 100, 'classification__solver': 'lbfgs'}

	Create pipeline with log ...
	Start grid search...
	Grid search done in 13.101s
	Get best model...
Pipeline(steps=[('variance_threshold', VarianceThreshold()),
                ('scaler', StandardScaler()),
                ('feature_selection', SelectKBest(k=30)),
                ('classification',
                 LogisticRegression(C=0.001, class_weight='balanced'))])
	Fit best model...
Val Acc: 0.5917329093799681 Best GS Acc: 0.569958100

0it [00:00, ?it/s]


	Create pipeline with log ...
	Start grid search...
	Grid search done in 13.264s
	Get best model...
Pipeline(steps=[('variance_threshold', VarianceThreshold()),
                ('scaler', StandardScaler()),
                ('feature_selection', SelectKBest(k=30)),
                ('classification',
                 LogisticRegression(C=0.001, class_weight='balanced'))])
	Fit best model...
Val Acc: 0.5213017269468883 Best GS Acc: 0.5538443789909021 Best Params: {'classification__C': 0.001, 'classification__max_iter': 100, 'classification__solver': 'lbfgs'}

	Create pipeline with log ...
	Start grid search...
	Grid search done in 13.110s
	Get best model...
Pipeline(steps=[('variance_threshold', VarianceThreshold()),
                ('scaler', StandardScaler()),
                ('feature_selection', SelectKBest(k=30)),
                ('classification',
                 LogisticRegression(C=0.001, class_weight='balanced'))])
	Fit best model...
Val Acc: 0.5914018231091401 Best GS Acc: 0.5

0it [00:00, ?it/s]


	Create pipeline with log ...
	Start grid search...
	Grid search done in 14.489s
	Get best model...
Pipeline(steps=[('variance_threshold', VarianceThreshold()),
                ('scaler', StandardScaler()),
                ('feature_selection', SelectKBest(k=30)),
                ('classification',
                 LogisticRegression(C=0.1, class_weight='balanced'))])
	Fit best model...
Val Acc: 0.5775745649573638 Best GS Acc: 0.5539248906710565 Best Params: {'classification__C': 0.1, 'classification__max_iter': 100, 'classification__solver': 'lbfgs'}

	Create pipeline with log ...
	Start grid search...
	Grid search done in 13.199s
	Get best model...
Pipeline(steps=[('variance_threshold', VarianceThreshold()),
                ('scaler', StandardScaler()),
                ('feature_selection', SelectKBest(k=30)),
                ('classification',
                 LogisticRegression(C=1, class_weight='balanced'))])
	Fit best model...
Val Acc: 0.5383987563155849 Best GS Acc: 0.601449651

In [34]:
print("LR with char ngrams")
classify_cv(log_charngrams_df, 'binary', small_clf_lst)

LR with char ngrams


  0%|          | 0/1 [00:00<?, ?it/s]

Classifier:  log 



  0%|          | 0/5 [00:00<?, ?it/s]


Trait to predict:  openness 



...


	Create pipeline with log ...
	Start grid search...
	Grid search done in 13.994s
	Get best model...
Pipeline(steps=[('variance_threshold', VarianceThreshold()),
                ('scaler', StandardScaler()),
                ('feature_selection', SelectKBest(k=30)),
                ('classification',
                 LogisticRegression(C=10, class_weight='balanced'))])
	Fit best model...
Val Acc: 0.5330084696600534 Best GS Acc: 0.5857819135346445 Best Params: {'classification__C': 10, 'classification__max_iter': 100, 'classification__solver': 'lbfgs'}

	Create pipeline with log ...
	Start grid search...
	Grid search done in 14.023s
	Get best model...
Pipeline(steps=[('variance_threshold', VarianceThreshold()),
                ('scaler', StandardScaler()),
                ('feature_selection', SelectKBest(k=30)),
                ('classification',
                 LogisticRegression(C=0.001, class_weight='balanced',
                                    solver='liblinear'))])
	Fit best mod

0it [00:00, ?it/s]


	Create pipeline with log ...
	Start grid search...
	Grid search done in 13.531s
	Get best model...
Pipeline(steps=[('variance_threshold', VarianceThreshold()),
                ('scaler', StandardScaler()),
                ('feature_selection', SelectKBest(k=30)),
                ('classification',
                 LogisticRegression(C=0.001, class_weight='balanced'))])
	Fit best model...
Val Acc: 0.506809620399884 Best GS Acc: 0.5309079454311204 Best Params: {'classification__C': 0.001, 'classification__max_iter': 100, 'classification__solver': 'lbfgs'}

	Create pipeline with log ...
	Start grid search...
	Grid search done in 13.796s
	Get best model...
Pipeline(steps=[('variance_threshold', VarianceThreshold()),
                ('scaler', StandardScaler()),
                ('feature_selection', SelectKBest(k=30)),
                ('classification',
                 LogisticRegression(C=0.01, class_weight='balanced'))])
	Fit best model...
Val Acc: 0.5499534445669275 Best GS Acc: 0.499

0it [00:00, ?it/s]


	Create pipeline with log ...
	Start grid search...
	Grid search done in 14.109s
	Get best model...
Pipeline(steps=[('variance_threshold', VarianceThreshold()),
                ('scaler', StandardScaler()),
                ('feature_selection', SelectKBest(k=30)),
                ('classification',
                 LogisticRegression(C=0.01, class_weight='balanced'))])
	Fit best model...
Val Acc: 0.518600497048773 Best GS Acc: 0.5399961669056991 Best Params: {'classification__C': 0.01, 'classification__max_iter': 100, 'classification__solver': 'lbfgs'}

	Create pipeline with log ...
	Start grid search...
	Grid search done in 15.440s
	Get best model...
Pipeline(steps=[('variance_threshold', VarianceThreshold()),
                ('scaler', StandardScaler()),
                ('feature_selection', SelectKBest(k=30)),
                ('classification',
                 LogisticRegression(C=0.001, class_weight='balanced'))])
	Fit best model...
Val Acc: 0.5605941459152469 Best GS Acc: 0.5450

In [35]:
print("LR with wordlists")
classify_cv(log_wordlists_df, 'binary', small_clf_lst)

LR with wordlists


  0%|          | 0/1 [00:00<?, ?it/s]

Classifier:  log 



  0%|          | 0/5 [00:00<?, ?it/s]


Trait to predict:  openness 



...


	Create pipeline with log ...
	Start grid search...
	Grid search done in 7.168s
	Get best model...
Pipeline(steps=[('variance_threshold', VarianceThreshold()),
                ('scaler', StandardScaler()),
                ('feature_selection', SelectKBest(k=30)),
                ('classification',
                 LogisticRegression(C=10000, class_weight='balanced'))])
	Fit best model...
Val Acc: 0.5043339015445012 Best GS Acc: 0.549911210175348 Best Params: {'classification__C': 10000, 'classification__max_iter': 100, 'classification__solver': 'lbfgs'}

	Create pipeline with log ...
	Start grid search...


/home/sophia/ma_py/.venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/sophia/ma_py/.venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-

	Grid search done in 11.666s
	Get best model...
Pipeline(steps=[('variance_threshold', VarianceThreshold()),
                ('scaler', StandardScaler()),
                ('feature_selection', SelectKBest(k=30)),
                ('classification',
                 LogisticRegression(C=10000, class_weight='balanced'))])
	Fit best model...
Val Acc: 0.5610816326530612 Best GS Acc: 0.5097067778384801 Best Params: {'classification__C': 10000, 'classification__max_iter': 100, 'classification__solver': 'lbfgs'}

	Create pipeline with log ...
	Start grid search...
	Grid search done in 8.382s
	Get best model...
Pipeline(steps=[('variance_threshold', VarianceThreshold()),
                ('scaler', StandardScaler()),
                ('feature_selection', SelectKBest(k=30)),
                ('classification',
                 LogisticRegression(C=10, class_weight='balanced',
                                    solver='liblinear'))])
	Fit best model...
Val Acc: 0.5374827502232324 Best GS Acc: 0.52

0it [00:00, ?it/s]


	Create pipeline with log ...
	Start grid search...
	Grid search done in 5.649s
	Get best model...
Pipeline(steps=[('variance_threshold', VarianceThreshold()),
                ('scaler', StandardScaler()),
                ('feature_selection', SelectKBest(k=30)),
                ('classification',
                 LogisticRegression(C=0.1, class_weight='balanced'))])
	Fit best model...
Val Acc: 0.5620061933840766 Best GS Acc: 0.5691438212239133 Best Params: {'classification__C': 0.1, 'classification__max_iter': 100, 'classification__solver': 'lbfgs'}

	Create pipeline with log ...
	Start grid search...
	Grid search done in 10.947s
	Get best model...
Pipeline(steps=[('variance_threshold', VarianceThreshold()),
                ('scaler', StandardScaler()),
                ('feature_selection', SelectKBest(k=30)),
                ('classification',
                 LogisticRegression(C=0.001, class_weight='balanced'))])
	Fit best model...
Val Acc: 0.46695864124998787 Best GS Acc: 0.57387

In [36]:
print("LR with posts features")
classify_cv(log_posts_df, 'binary', small_clf_lst)

LR with posts features


  0%|          | 0/1 [00:00<?, ?it/s]

Classifier:  log 



  0%|          | 0/5 [00:00<?, ?it/s]


Trait to predict:  openness 



...

/home/sophia/ma_py/.venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


Val Acc: 0.48602215682880384 Best GS Acc: 0.47771107426637754 Best Params: {'classification__C': 1, 'classification__max_iter': 100, 'classification__solver': 'saga'}

	Create pipeline with log ...
	Start grid search...
	Grid search done in 203.092s
	Get best model...
Pipeline(steps=[('variance_threshold', VarianceThreshold()),
                ('scaler', StandardScaler()),
                ('feature_selection', SelectKBest(k=30)),
                ('classification',
                 LogisticRegression(C=0.01, class_weight='balanced'))])
	Fit best model...
Val Acc: 0.4897064486507302 Best GS Acc: 0.4620835118696583 Best Params: {'classification__C': 0.01, 'classification__max_iter': 100, 'classification__solver': 'lbfgs'}

	Create pipeline with log ...
	Start grid search...


/home/sophia/ma_py/.venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


	Grid search done in 206.449s
	Get best model...
Pipeline(steps=[('variance_threshold', VarianceThreshold()),
                ('scaler', StandardScaler()),
                ('feature_selection', SelectKBest(k=30)),
                ('classification',
                 LogisticRegression(C=10, class_weight='balanced',
                                    max_iter=1000, solver='saga'))])
	Fit best model...
Val Acc: 0.4721308637701229 Best GS Acc: 0.49037936109891406 Best Params: {'classification__C': 10, 'classification__max_iter': 1000, 'classification__solver': 'saga'}
Average f1 macro score:  0.46611845823885734


/home/sophia/ma_py/.venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [37]:
print("LR with posts features but without subreddit")
classify_cv(log_postswithoutsubreddits_df, 'binary', small_clf_lst)

LR with posts features but without subreddit


  0%|          | 0/1 [00:00<?, ?it/s]

Classifier:  log 



  0%|          | 0/5 [00:00<?, ?it/s]


Trait to predict:  openness 



...

/home/sophia/ma_py/.venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/home/sophia/ma_py/.venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


	Grid search done in 10.087s
	Get best model...
Pipeline(steps=[('variance_threshold', VarianceThreshold()),
                ('scaler', StandardScaler()),
                ('feature_selection', SelectKBest(k=30)),
                ('classification',
                 LogisticRegression(C=1, class_weight='balanced',
                                    solver='saga'))])
	Fit best model...
Val Acc: 0.4826200093211124 Best GS Acc: 0.4969158133929102 Best Params: {'classification__C': 1, 'classification__max_iter': 100, 'classification__solver': 'saga'}

	Create pipeline with log ...
	Start grid search...
	Grid search done in 9.766s
	Get best model...
Pipeline(steps=[('variance_threshold', VarianceThreshold()),
                ('scaler', StandardScaler()),
                ('feature_selection', SelectKBest(k=30)),
                ('classification',
                 LogisticRegression(C=0.001, class_weight='balanced',
                                    max_iter=500, solver='saga'))])
	Fit best 

/home/sophia/ma_py/.venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


	Grid search done in 10.355s
	Get best model...
Pipeline(steps=[('variance_threshold', VarianceThreshold()),
                ('scaler', StandardScaler()),
                ('feature_selection', SelectKBest(k=30)),
                ('classification',
                 LogisticRegression(C=10, class_weight='balanced',
                                    max_iter=1000, solver='saga'))])
	Fit best model...
Val Acc: 0.5201327949056458 Best GS Acc: 0.48632133479752004 Best Params: {'classification__C': 10, 'classification__max_iter': 1000, 'classification__solver': 'saga'}
Average f1 macro score:  0.48302304931912127


/home/sophia/ma_py/.venv/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [ ]:
from sklearn.feature_selection import mutual_info_classif, SequentialFeatureSelector
from sklearn.decomposition import PCA

def get_classifier(classifier):
    if classifier == 'log':
        return LogisticRegression(class_weight='balanced')
    
def get_featureselection(fs, classifier):
    if fs == 'anova':
        return SelectKBest(f_classif, k=30)
    if fs == 'mutual':
        return mutual_info_classif()
    if fs == 'sequential':
        return SequentialFeatureSelector(get_classifier(classifier), n_features_to_select=30, n_jobs=-1)

    
def new_pipe(classifier, fs, dim):
    if dim:
        pipeline = Pipeline([
              ('variance_threshold', VarianceThreshold()),
              ('scaler', StandardScaler()),
              ('pca', PCA(n_components=100)),
              ('feature_selection',  get_featureselection(fs, classifier)),
              ('classification', get_classifier(classifier))
            ])
    else:
        pipeline = Pipeline([
              ('variance_threshold', VarianceThreshold()),
              ('scaler', StandardScaler()),
              ('feature_selection',  get_featureselection(fs, classifier)),
              ('classification', get_classifier(classifier))
            ])
    return pipeline

def classify(df, classifier, classes, fs, dim):
    for trait_name in traits:
        print("Trait to predict: ", trait_name)
        x,y = trait(df, classes,trait_name)
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)
        clf = new_pipe(classifier, fs, dim)
        clf.fit(x_train, y_train)
        y_pred=clf.predict(x_test)
        report = scores(y_test, y_pred, "report")
        print(report)

classify(log_charngrams_df, 'log', classes='binary', fs='mutual', dim=True)

## Histogram of true traits

In [ ]:
def check_imbalance(df, traits):
    length = len(df)
    o = df['trait', 'big5_o']
    c = df['trait', 'big5_c']
    e = df['trait', 'big5_e']
    a = df['trait', 'big5_a']
    n = df['trait', 'big5_n']
    binarylst = [o, c, e, a, n]
    o5 = df['trait', 'big5_o_multi']
    c5 = df['trait', 'big5_c_multi']
    e5 = df['trait', 'big5_e_multi']
    a5 = df['trait', 'big5_a_multi']
    n5 = df['trait', 'big5_n_multi']
    multilst = [o5, c5, e5, a5, n5]
    
    result = []
    for trait in binarylst: 
        result.append(np.bincount(trait) / length)
    result5 = []
    for trait in multilst:
        result5.append(np.bincount(trait) / len(trait))
    
    print("Distribution of the true trait values in the classes (in %):\n")
    for i in range(len(traits)):
        print(traits[i], "\n\tBinary: ", result[i], "\n\t5 classes: ", result5[i], "\n")
    
#     result =np.bincount(o) / len(o)
#     result5 =np.bincount(o5) / len(o)
#     print("Openness\n\tBinary: ", result, "\n\t5 classes: ", result5)

    
check_imbalance(df, traits)

In [ ]:
# true histogram plots
all_hist_true(df)

In [ ]:
# %%javascript
# IPython.OutputArea.auto_scroll_threshold = 1000;

### check results on train set with k nearest neighbor algorithm

In [ ]:
# classify(df, 'knn', 'binary', trainscores=True, plotting=False, weighted=False, detailed=False)

In [ ]:
# classify(df, 'knn', 'multi', trainscores=True, plotting=False, weighted=False, detailed=False)

### Logistic regression

In [ ]:
# classify(df, 'log', 'binary', plotting=True)

In [ ]:
# classify(df, 'log', 'binary', plotting=True, weighted=True)

### Logistic Regression: multiclass

In [ ]:
# classify(df, 'multilog', 'multi', plotting=True, weighted=True, detailed=False)

### MLP

In [ ]:
# classify(df, 'mlp', 'binary', plotting=True)

In [ ]:
# classify(df, 'mlp', 'multi', plotting=True)

### SVM

In [ ]:
# classify(df, 'svm', 'binary', plotting=True)

In [ ]:
# classify(df, 'svm', 'multi', plotting=True)

In [ ]:
# classify(df, 'svmlinear', 'linear', plotting=True)

### Linear Regression

In [ ]:
# classify(df, 'linear', 'linear', plotting=True)

# Results without predictor subreddit

###  binary log (without subreddit)

In [ ]:
# classify(smalldf, 'log', 'binary', plotting=True, weighted=True, detailed=False)

### multi log (without subreddit)

In [ ]:
# classify(smalldf, 'multilog', 'multi', plotting=True, weighted=True, detailed=False)

### MLP (without subreddit)

In [ ]:
# classify(smalldf, 'mlp', 'binary', plotting=True, weighted=False, detailed=False)

### Linear Regression (without subreddit)

In [ ]:
# classify(smalldf, 'lin', 'linear', plotting=True)